In [11]:
import sqlite3
from IPython.display import display
import pandas as pd
from tabulate import tabulate

In [12]:
!rm -rf data
!cp -r ../Scrapers/data .
!cp ../Scrapers/nfl.db data/

In [ ]:
# Get 2024 QB's

# Load the provided NFL database
db_path = 'data/nfl.db'

# Connect to the database
conn = sqlite3.connect(db_path)

# Get the list of tables to understand the structure of the database
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()

# Display the list of tables
tables

# Now that we understand the structure, let's search for the quarterbacks currently listed for the Packers and Titans in the latest season.
# We will focus on players with the position listed as "QB" to verify starting QBs and any injury statuses.

# Packers QB roster query
packers_qb_query = """
    SELECT full_name, position, status
    FROM Rosters
    WHERE team = 'GB' AND position = 'QB' AND season = 2024;
"""

# Titans QB roster query
titans_qb_query = """
    SELECT full_name, position, status
    FROM Rosters
    WHERE team = 'TEN' AND position = 'QB' AND season = 2024;
"""

# Execute both queries
cursor.execute(packers_qb_query)
packers_qbs = cursor.fetchall()

cursor.execute(titans_qb_query)
titans_qbs = cursor.fetchall()

packers_qbs, titans_qbs  # Display the current QBs and their statuses for both teams


In [13]:
# Single teams defensive stats

def calculate_team_total_yards_defensive_stats(team_abbreviation, season, data_df):
    # Filter for the specified season by extracting the season from the 'game_id' column
    data_df['season'] = data_df['game_id'].str.slice(0, 4).astype(int)

    # Filter the data for the specific season
    season_games = data_df[data_df['season'] == season]

    # Initialize lists for away and home games
    away_games = []
    home_games = []

    # Loop through each game and split game_id to determine home/away status
    for index, row in season_games.iterrows():
        game_id_parts = row['game_id'].split('_')
        game_season, week, away_team, home_team = game_id_parts

        # Convert the week to an integer to filter weeks 01-18
        week_num = int(week)

        if week_num > 18:
            continue  # Skip postseason weeks

        if away_team == team_abbreviation:
            away_games.append(row)  # The team was the away team in this game
        elif home_team == team_abbreviation:
            home_games.append(row)  # The team was the home team in this game

    # Convert lists to DataFrames for easier processing
    away_games_df = pd.DataFrame(away_games)
    home_games_df = pd.DataFrame(home_games)

    # For home games, we want the stats from the "away" columns (opponent is away team)
    home_defense_stats = home_games_df[['away_rush_yds', 'away_pass_yds']].sum()

    # For away games, we want the stats from the "home" columns (opponent is home team)
    away_defense_stats = away_games_df[['home_rush_yds', 'home_pass_yds']].sum()

    # Combine rushing and passing yards for all games
    total_rush_yards_allowed = home_defense_stats['away_rush_yds'] + away_defense_stats['home_rush_yds']
    total_pass_yards_allowed = home_defense_stats['away_pass_yds'] + away_defense_stats['home_pass_yds']

    # Calculate total yards allowed (rushing + passing)
    total_yards_allowed = total_rush_yards_allowed + total_pass_yards_allowed

    # Number of games played by the team
    num_games = len(home_games_df) + len(away_games_df)

    defensive_summary = {
        "avg_total_yards_allowed": total_yards_allowed / num_games if num_games > 0 else 0,
        "avg_rush_yards_allowed": total_rush_yards_allowed / num_games if num_games > 0 else 0,
        "avg_pass_yards_allowed": total_pass_yards_allowed / num_games if num_games > 0 else 0
    }

    return defensive_summary

# Load the dataset (assuming it's located in 'data/' folder)
file_path = 'data/all_team_game_logs.csv'
team_game_logs_df = pd.read_csv(file_path)

# Example usage: calculate total yards allowed for the Ravens ('BAL') and Cowboys ('DAL') for the 2023 season
ravens_defensive_stats_2023 = calculate_team_total_yards_defensive_stats('BAL', 2023, team_game_logs_df)
cowboys_defensive_stats_2023 = calculate_team_total_yards_defensive_stats('DAL', 2023, team_game_logs_df)

# Output the defensive stats for both teams
print("Ravens 2023 Defensive Total Yards Stats:", ravens_defensive_stats_2023)
print("Cowboys 2023 Defensive Total Yards Stats:", cowboys_defensive_stats_2023)

Ravens 2023 Defensive Total Yards Stats: {'avg_total_yards_allowed': 301.3529411764706, 'avg_rush_yards_allowed': 109.41176470588235, 'avg_pass_yards_allowed': 191.94117647058823}
Cowboys 2023 Defensive Total Yards Stats: {'avg_total_yards_allowed': 299.70588235294116, 'avg_rush_yards_allowed': 112.3529411764706, 'avg_pass_yards_allowed': 187.35294117647058}


In [17]:
# Loop and rank all teams 2023 season only ^

# Function to calculate total defensive yards allowed for a given team and season
def calculate_team_total_yards_defensive_stats(team_abbreviation, season, data_df):
    # Filter for the specified season by extracting the season from the 'game_id' column
    data_df['season'] = data_df['game_id'].str.slice(0, 4).astype(int)

    # Filter the data for the specific season
    season_games = data_df[data_df['season'] == season]
    
    # Filter for unplayed games where the game_id contains "2024" and both home and away points are either NaN or 0
    unplayed_games = season_games[
        season_games['game_id'].str.contains('2024') &  # Check if 'game_id' contains "2024"
        ((season_games['home_pts_off'].isnull() | (season_games['home_pts_off'] == 0)) &
         (season_games['away_pts_off'].isnull() | (season_games['away_pts_off'] == 0)))
    ]
    
    # Create a list of unplayed game_ids
    unplayed_game_ids = unplayed_games['game_id'].tolist()
    
    # Output the list of unplayed game_ids
    print("List of unplayed 2024 game_ids:")
    print(unplayed_game_ids)
    
    # Filter out the unplayed games from the original DataFrame
    season_games = season_games[~season_games['game_id'].isin(unplayed_game_ids)]

    # Initialize lists for away and home games
    away_games = []
    home_games = []

    # Loop through each game and split game_id to determine home/away status
    for index, row in season_games.iterrows():
        game_id_parts = row['game_id'].split('_')
        game_season, week, away_team, home_team = game_id_parts

        # Convert the week to an integer to filter weeks 01-18
        week_num = int(week)

        if week_num > 18:
            continue  # Skip postseason weeks

        if away_team == team_abbreviation:
            away_games.append(row)  # The team was the away team in this game
        elif home_team == team_abbreviation:
            home_games.append(row)  # The team was the home team in this game

    # Convert lists to DataFrames for easier processing
    away_games_df = pd.DataFrame(away_games)
    home_games_df = pd.DataFrame(home_games)

    # For home games, we want the stats from the "away" columns (opponent is away team)
    home_defense_stats = home_games_df[['away_rush_yds', 'away_pass_yds']].sum()

    # For away games, we want the stats from the "home" columns (opponent is home team)
    away_defense_stats = away_games_df[['home_rush_yds', 'home_pass_yds']].sum()

    # Combine rushing and passing yards for all games
    total_rush_yards_allowed = home_defense_stats['away_rush_yds'] + away_defense_stats['home_rush_yds']
    total_pass_yards_allowed = home_defense_stats['away_pass_yds'] + away_defense_stats['home_pass_yds']

    # Calculate total yards allowed (rushing + passing)
    total_yards_allowed = total_rush_yards_allowed + total_pass_yards_allowed

    # Number of games played by the team
    num_games = len(home_games_df) + len(away_games_df)

    # Calculate averages for total yards allowed
    defensive_summary = {
        "avg_total_yards_allowed": total_yards_allowed / num_games if num_games > 0 else 0,
        "avg_rush_yards_allowed": total_rush_yards_allowed / num_games if num_games > 0 else 0,
        "avg_pass_yards_allowed": total_pass_yards_allowed / num_games if num_games > 0 else 0
    }

    return defensive_summary

# Load the dataset (assuming it's located in 'data/' folder)
file_path = 'data/all_team_game_logs.csv'
team_game_logs_df = pd.read_csv(file_path)

# List of all team abbreviations
teams = ['ARI', 'ATL', 'BAL', 'BUF', 'CAR', 'CHI', 'CIN', 'CLE', 'DAL', 'DEN', 
         'DET', 'GB', 'HOU', 'IND', 'JAX', 'KC', 'LAC', 'LAR', 'LVR', 'MIA', 
         'MIN', 'NE', 'NO', 'NYG', 'NYJ', 'PHI', 'PIT', 'SEA', 'SF', 'TB', 'TEN', 'WAS']

# List to hold defensive stats for all teams
defensive_stats_list = []

# Calculate defensive stats for all teams
for team in teams:
    # stats = calculate_team_total_yards_defensive_stats(team, 2023, team_game_logs_df)
    stats = calculate_team_total_yards_defensive_stats(team, 2024, team_game_logs_df)
    stats['team'] = team  # Add team abbreviation to the stats
    defensive_stats_list.append(stats)

# Create a DataFrame from the defensive stats list
defensive_stats_df = pd.DataFrame(defensive_stats_list)

# Insert 'team' column at the first position
defensive_stats_df.insert(0, 'team', defensive_stats_df.pop('team'))

# Sort the DataFrame by average total yards allowed in descending order
sorted_defensive_stats_df = defensive_stats_df.sort_values(by='avg_total_yards_allowed', ascending=False)

# Reset index and drop the old index
sorted_defensive_stats_df.reset_index(drop=True, inplace=True)

# Add a new column for row numbers
sorted_defensive_stats_df.insert(0, 'No', sorted_defensive_stats_df.index + 1)

# Save to CSV without an index
sorted_defensive_stats_df.to_csv(f'data/team_defense_analysis_{season}.csv', index=False)

# Display the sorted DataFrame using Tabulate
print(tabulate(sorted_defensive_stats_df, headers='keys', tablefmt='psql', showindex=False))

# Print confirmation message
print(f"Defensive stats for all teams have been saved to 'team_defense_analysis_{season}.csv'.")


List of unplayed 2024 game_ids:
['2024_03_BAL_DAL', '2024_03_CAR_LVR', '2024_03_CHI_IND', '2024_03_DEN_TB', '2024_03_DET_ARI', '2024_03_GB_TEN', '2024_03_HOU_MIN', '2024_03_JAX_BUF', '2024_03_KC_ATL', '2024_03_LAC_PIT', '2024_03_MIA_SEA', '2024_03_NE_NYJ', '2024_03_NYG_CLE', '2024_03_PHI_NO', '2024_03_SF_LAR', '2024_03_WAS_CIN', '2024_04_BUF_BAL', '2024_04_CIN_CAR', '2024_04_CLE_LVR', '2024_04_DAL_NYG', '2024_04_DEN_NYJ', '2024_04_JAX_HOU', '2024_04_KC_LAC', '2024_04_LAR_CHI', '2024_04_MIN_GB', '2024_04_NE_SF', '2024_04_NO_ATL', '2024_04_PHI_TB', '2024_04_PIT_IND', '2024_04_SEA_DET', '2024_04_TEN_MIA', '2024_04_WAS_ARI', '2024_05_ARI_SF', '2024_05_BAL_CIN', '2024_05_BUF_HOU', '2024_05_CAR_CHI', '2024_05_CLE_WAS', '2024_05_DAL_PIT', '2024_05_GB_LAR', '2024_05_IND_JAX', '2024_05_LVR_DEN', '2024_05_MIA_NE', '2024_05_NO_KC', '2024_05_NYG_SEA', '2024_05_NYJ_MIN', '2024_05_TB_ATL', '2024_06_ARI_GB', '2024_06_ATL_CAR', '2024_06_BUF_NYJ', '2024_06_CIN_NYG', '2024_06_CLE_PHI', '2024_06_DET_DAL'

KeyError: "None of [Index(['home_rush_yds', 'home_pass_yds'], dtype='object')] are in the [columns]"

In [25]:
# Loop and rank all teams 2020-2023 seasons ^

# Function to calculate total defensive yards allowed for a given team and season
def calculate_team_total_yards_defensive_stats(team_abbreviation, season, data_df):
    # Filter for the specified season by extracting the season from the 'game_id' column
    data_df['season'] = data_df['game_id'].str.slice(0, 4).astype(int)

    # Filter the data for the specific season
    if season == 2024:
        # For the current season, only consider games played so far
        current_week = data_df[data_df['season'] == season]['game_id'].str.slice(5, 7).astype(int).max()  # Get the latest week
        season_games = data_df[(data_df['season'] == season) & (data_df['game_id'].str.slice(5, 7).astype(int) <= current_week)]
    else:
        season_games = data_df[data_df['season'] == season]

    # Initialize lists for away and home games
    away_games = []
    home_games = []

    # Loop through each game and split game_id to determine home/away status
    for index, row in season_games.iterrows():
        game_id_parts = row['game_id'].split('_')
        game_season, week, away_team, home_team = game_id_parts

        # Convert the week to an integer to filter weeks 01-18
        week_num = int(week)

        if week_num > 18:
            continue  # Skip postseason weeks

        if away_team == team_abbreviation:
            away_games.append(row)  # The team was the away team in this game
        elif home_team == team_abbreviation:
            home_games.append(row)  # The team was the home team in this game

    # Convert lists to DataFrames for easier processing
    away_games_df = pd.DataFrame(away_games)
    home_games_df = pd.DataFrame(home_games)

    # For home games, we want the stats from the "away" columns (opponent is away team)
    home_defense_stats = home_games_df[['away_rush_yds', 'away_pass_yds']].sum()

    # For away games, we want the stats from the "home" columns (opponent is home team)
    away_defense_stats = away_games_df[['home_rush_yds', 'home_pass_yds']].sum()

    # Combine rushing and passing yards for all games
    total_rush_yards_allowed = home_defense_stats['away_rush_yds'] + away_defense_stats['home_rush_yds']
    total_pass_yards_allowed = home_defense_stats['away_pass_yds'] + away_defense_stats['home_pass_yds']

    # Calculate total yards allowed (rushing + passing)
    total_yards_allowed = total_rush_yards_allowed + total_pass_yards_allowed

    # Number of games played by the team
    num_games = len(home_games_df) + len(away_games_df)

    # Calculate averages for total yards allowed
    defensive_summary = {
        "avg_total_yards_allowed": total_yards_allowed / num_games if num_games > 0 else 0,
        "avg_rush_yards_allowed": total_rush_yards_allowed / num_games if num_games > 0 else 0,
        "avg_pass_yards_allowed": total_pass_yards_allowed / num_games if num_games > 0 else 0,
        "season": season
    }

    return defensive_summary

# Load the dataset (assuming it's located in 'data/' folder)
file_path = 'data/all_team_game_logs.csv'
team_game_logs_df = pd.read_csv(file_path)

# List of all team abbreviations
teams = ['ARI', 'ATL', 'BAL', 'BUF', 'CAR', 'CHI', 'CIN', 'CLE', 'DAL', 'DEN', 
         'DET', 'GB', 'HOU', 'IND', 'JAX', 'KC', 'LAC', 'LAR', 'LVR', 'MIA', 
         'MIN', 'NE', 'NO', 'NYG', 'NYJ', 'PHI', 'PIT', 'SEA', 'SF', 'TB', 'TEN', 'WAS']

# List to hold defensive stats for all teams
defensive_stats_list = []

# Loop through the seasons 2020 to 2023, and the played games in 2024
for season in range(2020, 2024):
    for team in teams:
        stats = calculate_team_total_yards_defensive_stats(team, season, team_game_logs_df)
        stats['team'] = team  # Add team abbreviation to the stats
        defensive_stats_list.append(stats)

# Handle the 2024 season for only played games
for team in teams:
    stats = calculate_team_total_yards_defensive_stats(team, 2024, team_game_logs_df)
    stats['team'] = team  # Add team abbreviation to the stats
    defensive_stats_list.append(stats)

# Create a DataFrame from the defensive stats list
defensive_stats_df = pd.DataFrame(defensive_stats_list)

# Insert 'team' column at the first position
defensive_stats_df.insert(0, 'team', defensive_stats_df.pop('team'))

# Sort the DataFrame by average total yards allowed in descending order
# sorted_defensive_stats_df = defensive_stats_df.sort_values(by='avg_total_yards_allowed', ascending=False)
sorted_defensive_stats_df = defensive_stats_df.sort_values(by=['season'], ascending=[True])

# Reset index and drop the old index
sorted_defensive_stats_df.reset_index(drop=True, inplace=True)

# Add a new column for row numbers
sorted_defensive_stats_df.insert(0, 'No', sorted_defensive_stats_df.index + 1)

# Save to CSV without an index
sorted_defensive_stats_df.to_csv('data/team_defense_analysis_2020_2023.csv', index=False)

# Display the sorted DataFrame using Tabulate
# print(tabulate(sorted_defensive_stats_df, headers='keys', tablefmt='psql', showindex=False))

# Print confirmation message
print("Defensive stats for all teams have been saved to 'team_defense_analysis_2020_2023.csv'.")

# Read the saved CSV file
final_df = pd.read_csv('data/team_defense_analysis_2020_2023.csv')

# Filter the DataFrame to include only the 2024 season
df_2023 = final_df[final_df['season'] == 2023].copy()

# Reset index for the filtered DataFrame (optional, for cleaner display)
df_2023.reset_index(drop=True, inplace=True)

# Add a new 'No' column specific to the 2024 data (optional)
# df_2024.insert(0, 'No', df_2024.index + 1)

# Display only the 2024 DataFrame using Tabulate
print('\n                                                2023 STATS')
print(tabulate(df_2023, headers='keys', tablefmt='psql', showindex=False))

# Print confirmation message
print("Defensive stats for all teams have been saved to 'team_defense_analysis_2020_2023.csv'.")

Defensive stats for all teams have been saved to 'team_defense_analysis_2020_2023.csv'.

                                                2023 STATS
+------+--------+---------------------------+--------------------------+--------------------------+----------+
|   No | team   |   avg_total_yards_allowed |   avg_rush_yards_allowed |   avg_pass_yards_allowed |   season |
|------+--------+---------------------------+--------------------------+--------------------------+----------|
|   97 | LVR    |                   330.882 |                 118.529  |                  212.353 |     2023 |
|   98 | MIA    |                   318.294 |                  97.0588 |                  221.235 |     2023 |
|   99 | MIN    |                   333.176 |                  98.7059 |                  234.471 |     2023 |
|  100 | NE     |                   301.588 |                  93.1765 |                  208.412 |     2023 |
|  101 | NO     |                   327.176 |                 119.882  |   

In [5]:
import pandas as pd
from tabulate import tabulate

# Function to calculate total defensive yards allowed for a given team and season
def calculate_team_total_yards_defensive_stats(team_abbreviation, season, data_df):
    # Filter for the specified season by extracting the season from the 'game_id' column
    data_df['season'] = data_df['game_id'].str.slice(0, 4).astype(int)

    # Filter the data for the specific season
    if season == 2024:
        # For the current season, only consider games played so far
        current_week = data_df[data_df['season'] == season]['game_id'].str.slice(5, 7).astype(int).max()  # Get the latest week
        season_games = data_df[(data_df['season'] == season) & (data_df['game_id'].str.slice(5, 7).astype(int) <= current_week)]
    else:
        season_games = data_df[data_df['season'] == season]

    # Initialize lists for away and home games
    away_games = []
    home_games = []

    # Loop through each game and split game_id to determine home/away status
    for index, row in season_games.iterrows():
        game_id_parts = row['game_id'].split('_')
        game_season, week, away_team, home_team = game_id_parts

        # Convert the week to an integer to filter weeks 01-18
        week_num = int(week)

        if week_num > 18:
            continue  # Skip postseason weeks

        if away_team == team_abbreviation:
            away_games.append(row)  # The team was the away team in this game
        elif home_team == team_abbreviation:
            home_games.append(row)  # The team was the home team in this game

    # Convert lists to DataFrames for easier processing
    away_games_df = pd.DataFrame(away_games)
    home_games_df = pd.DataFrame(home_games)

    # For home games, we want the stats from the "away" columns (opponent is away team)
    home_defense_stats = home_games_df[['away_rush_yds', 'away_pass_yds']].sum()

    # For away games, we want the stats from the "home" columns (opponent is home team)
    away_defense_stats = away_games_df[['home_rush_yds', 'home_pass_yds']].sum()

    # Combine rushing and passing yards for all games
    total_rush_yards_allowed = home_defense_stats['away_rush_yds'] + away_defense_stats['home_rush_yds']
    total_pass_yards_allowed = home_defense_stats['away_pass_yds'] + away_defense_stats['home_pass_yds']

    # Calculate total yards allowed (rushing + passing)
    total_yards_allowed = total_rush_yards_allowed + total_pass_yards_allowed

    # Number of games played by the team
    num_games = len(home_games_df) + len(away_games_df)

    # Calculate averages for total yards allowed
    defensive_summary = {
        "avg_total_yards_allowed": total_yards_allowed / num_games if num_games > 0 else 0,
        "avg_rush_yards_allowed": total_rush_yards_allowed / num_games if num_games > 0 else 0,
        "avg_pass_yards_allowed": total_pass_yards_allowed / num_games if num_games > 0 else 0,
        "season": season
    }

    return defensive_summary

# Load the dataset (assuming it's located in 'data/' folder)
file_path = 'data/all_team_game_logs.csv'
team_game_logs_df = pd.read_csv(file_path)

# List of all team abbreviations
teams = ['ARI', 'ATL', 'BAL', 'BUF', 'CAR', 'CHI', 'CIN', 'CLE', 'DAL', 'DEN', 
         'DET', 'GB', 'HOU', 'IND', 'JAX', 'KC', 'LAC', 'LAR', 'LVR', 'MIA', 
         'MIN', 'NE', 'NO', 'NYG', 'NYJ', 'PHI', 'PIT', 'SEA', 'SF', 'TB', 'TEN', 'WAS']

# List to hold defensive stats for all teams
defensive_stats_list = []

# Loop through the seasons 2020 to 2024 (inclusive)
for season in range(2020, 2025):
    for team in teams:
        stats = calculate_team_total_yards_defensive_stats(team, season, team_game_logs_df)
        stats['team'] = team  # Add team abbreviation to the stats
        defensive_stats_list.append(stats)

# Note: Removed the separate loop for the 2024 season to avoid duplication

# Create a DataFrame from the defensive stats list
defensive_stats_df = pd.DataFrame(defensive_stats_list)

# Insert 'team' column at the first position
defensive_stats_df.insert(0, 'team', defensive_stats_df.pop('team'))

# Sort the DataFrame by 'team' and then by 'season' in ascending order
sorted_defensive_stats_df = defensive_stats_df.sort_values(by=['team', 'season'], ascending=[True, True])

# Reset index and drop the old index
sorted_defensive_stats_df.reset_index(drop=True, inplace=True)

# Add a new column for row numbers (optional)
sorted_defensive_stats_df.insert(0, 'No', sorted_defensive_stats_df.index + 1)

# Save the full sorted DataFrame to CSV
sorted_defensive_stats_df.to_csv('data/team_defense_analysis_2020_2024.csv', index=False)

# Read the saved CSV file
final_df = pd.read_csv('data/team_defense_analysis_2020_2024.csv')

# Filter the DataFrame to include only the 2024 season
df_2024 = final_df[final_df['season'] == 2024].copy()

# Reset index for the filtered DataFrame (optional, for cleaner display)
df_2024.reset_index(drop=True, inplace=True)

# Add a new 'No' column specific to the 2024 data (optional)
# df_2024.insert(0, 'No', df_2024.index + 1)

# Display only the 2024 DataFrame using Tabulate
print(tabulate(df_2024, headers='keys', tablefmt='psql', showindex=False))

# Print confirmation message
print("Defensive stats for all teams have been saved to 'team_defense_analysis_2020_2024.csv'.")


+------+--------+---------------------------+--------------------------+--------------------------+----------+
|   No | team   |   avg_total_yards_allowed |   avg_rush_yards_allowed |   avg_pass_yards_allowed |   season |
|------+--------+---------------------------+--------------------------+--------------------------+----------|
|    5 | ARI    |                   35.1176 |                 10.7647  |                  24.3529 |     2024 |
|   10 | ATL    |                   37.3529 |                 19       |                  18.3529 |     2024 |
|   15 | BAL    |                   36.0588 |                  5.82353 |                  30.2353 |     2024 |
|   20 | BUF    |                   36.5294 |                 15.4706  |                  21.0588 |     2024 |
|   25 | CAR    |                   42.8235 |                 23.4706  |                  19.3529 |     2024 |
|   30 | CHI    |                   32.5882 |                 12.6471  |                  19.9412 |     2024 |
|

In [ ]:
!open data/team_defense_analysis_2020_2024.csv

In [26]:
# Loop through all teams 2024 season ^

# Function to calculate total defensive yards allowed for a given team and season
def calculate_team_total_yards_defensive_stats(team_abbreviation, season, data_df):
    # Filter for the specified season by extracting the season from the 'game_id' column
    data_df['season'] = data_df['game_id'].str.slice(0, 4).astype(int)

    # Filter for unplayed games where the game_id contains "2024" and both home and away points are either NaN or 0
    unplayed_games = data_df[
        data_df['game_id'].str.contains('2024') &  # Check if 'game_id' contains "2024"
        ((data_df['home_pts_off'].isnull() | (data_df['home_pts_off'] == 0)) &
         (data_df['away_pts_off'].isnull() | (data_df['away_pts_off'] == 0)))
    ]
    
    # Create a list of unplayed game_ids
    unplayed_game_ids = unplayed_games['game_id'].tolist()
    
    # Output the list of unplayed game_ids
    # print("List of unplayed 2024 game_ids:")
    # print(unplayed_game_ids)

    # Filter the data for the specific season
    if season == 2024:
        # For the current season, only consider games played so far
        current_week = data_df[data_df['season'] == season]['game_id'].str.slice(5, 7).astype(int).max()  # Get the latest week
        season_games = data_df[(data_df['season'] == season) & (data_df['game_id'].str.slice(5, 7).astype(int) <= current_week)]
    else:
        season_games = data_df[data_df['season'] == season]

    # Initialize lists for away and home games
    away_games = []
    home_games = []

    # Loop through each game and split game_id to determine home/away status
    for index, row in season_games.iterrows():
        game_id_parts = row['game_id'].split('_')
        game_season, week, away_team, home_team = game_id_parts

        # Convert the week to an integer to filter weeks 01-18
        week_num = int(week)

        if week_num > 18:
            continue  # Skip postseason weeks

        if away_team == team_abbreviation:
            away_games.append(row)  # The team was the away team in this game
        elif home_team == team_abbreviation:
            home_games.append(row)  # The team was the home team in this game

    # Convert lists to DataFrames for easier processing
    away_games_df = pd.DataFrame(away_games)
    home_games_df = pd.DataFrame(home_games)

    # Optional: drop unplayed games from away_games_df and home_games_df
    away_games_df = away_games_df[~away_games_df['game_id'].isin(unplayed_game_ids)]
    home_games_df = home_games_df[~home_games_df['game_id'].isin(unplayed_game_ids)]

    # For home games, we want the stats from the "away" columns (opponent is away team)
    home_defense_stats = home_games_df[['away_rush_yds', 'away_pass_yds']].sum()

    # For away games, we want the stats from the "home" columns (opponent is home team)
    away_defense_stats = away_games_df[['home_rush_yds', 'home_pass_yds']].sum()

    # Combine rushing and passing yards for all games
    total_rush_yards_allowed = home_defense_stats['away_rush_yds'] + away_defense_stats['home_rush_yds']
    total_pass_yards_allowed = home_defense_stats['away_pass_yds'] + away_defense_stats['home_pass_yds']

    # Calculate total yards allowed (rushing + passing)
    total_yards_allowed = total_rush_yards_allowed + total_pass_yards_allowed

    # Number of games played by the team
    num_games = len(home_games_df) + len(away_games_df)

    # Calculate averages for total yards allowed
    defensive_summary = {
        "avg_total_yards_allowed": total_yards_allowed / num_games if num_games > 0 else 0,
        "avg_rush_yards_allowed": total_rush_yards_allowed / num_games if num_games > 0 else 0,
        "avg_pass_yards_allowed": total_pass_yards_allowed / num_games if num_games > 0 else 0
    }

    return defensive_summary

# Load the dataset (assuming it's located in 'data/' folder)
file_path = 'data/all_team_game_logs.csv'
team_game_logs_df = pd.read_csv(file_path)

# List of all team abbreviations
teams = ['ARI', 'ATL', 'BAL', 'BUF', 'CAR', 'CHI', 'CIN', 'CLE', 'DAL', 'DEN', 
         'DET', 'GB', 'HOU', 'IND', 'JAX', 'KC', 'LAC', 'LAR', 'LVR', 'MIA', 
         'MIN', 'NE', 'NO', 'NYG', 'NYJ', 'PHI', 'PIT', 'SEA', 'SF', 'TB', 'TEN', 'WAS']

# List to hold defensive stats for all teams
defensive_stats_list = []

# Loop through the seasons 2020 to 2024
for season in range(2024, 2025):
    for team in teams:
        # Calculate defensive stats
        stats = calculate_team_total_yards_defensive_stats(team, season, team_game_logs_df)
        stats['team'] = team  # Add team abbreviation to the stats
        
        # For the 2024 season, add the extra check for zero passing yards
        if season == 2024:
            if stats['avg_pass_yards_allowed'] > 0:
                defensive_stats_list.append(stats)
        else:
            defensive_stats_list.append(stats)

# Create a DataFrame from the defensive stats list
defensive_stats_df = pd.DataFrame(defensive_stats_list)

# Remove duplicate rows based on the 'team' column
defensive_stats_df = defensive_stats_df.drop_duplicates(subset='team', keep='first')

# Insert 'team' column at the first position
defensive_stats_df.insert(0, 'team', defensive_stats_df.pop('team'))

# Sort the DataFrame by average total yards allowed in descending order
sorted_defensive_stats_df = defensive_stats_df.sort_values(by='avg_total_yards_allowed', ascending=False)

# Reset index and drop the old index
sorted_defensive_stats_df.reset_index(drop=True, inplace=True)

# Add a new column for row numbers
sorted_defensive_stats_df.insert(0, 'No', sorted_defensive_stats_df.index + 1)

# Save to CSV without an index
sorted_defensive_stats_df.to_csv('data/team_defense_analysis_2020_2024.csv', index=False)

# Display the sorted DataFrame using Tabulate
print(tabulate(sorted_defensive_stats_df, headers='keys', tablefmt='psql', showindex=False))

# Print confirmation message
print("Defensive stats for all teams have been saved to 'data/team_defense_analysis_2020_2024.csv'.")

+------+--------+---------------------------+--------------------------+--------------------------+
|   No | team   |   avg_total_yards_allowed |   avg_rush_yards_allowed |   avg_pass_yards_allowed |
|------+--------+---------------------------+--------------------------+--------------------------|
|    1 | LAR    |                     426   |                    197   |                    229   |
|    2 | IND    |                     400   |                    237   |                    163   |
|    3 | PHI    |                     399.5 |                    157.5 |                    242   |
|    4 | KC     |                     386   |                    129.5 |                    256.5 |
|    5 | TB     |                     381   |                    138.5 |                    242.5 |
|    6 | GB     |                     374   |                    142   |                    232   |
|    7 | NYG    |                     368.5 |                    163   |                    205.5 |


In [27]:
# Underdog games and record in them ATS and straight up

# Function to get underdog games
def get_underdog_games(team_name, seasons, db_path):
    # Connect to the database
    conn = sqlite3.connect(db_path)
    
    # Prepare a query to get all games for the given team where they were the underdog
    query = f"""
    SELECT season, week, away_team, home_team, spread_line, team_favorite, result, home_score, away_score 
    FROM Games
    WHERE season IN ({','.join(map(str, seasons))})
    AND (home_team = '{team_name}' OR away_team = '{team_name}')
    AND team_favorite != '{team_name}';
    """
    
    # Execute query
    games = conn.execute(query).fetchall()
    conn.close()
    
    # Create a DataFrame
    columns = ['season', 'week', 'away_team', 'home_team', 'spread_line', 'team_favorite', 'result', 'home_score', 'away_score']
    games_df = pd.DataFrame(games, columns=columns)
    
    # Calculate the point differential and whether the team covered the spread
    games_df['point_differential'] = games_df.apply(lambda row: row['away_score'] - row['home_score'] if row['away_team'] == team_name else row['home_score'] - row['away_score'], axis=1)
    games_df['ats_differential'] = games_df.apply(lambda row: (row['point_differential'] + float(row['spread_line'])) if row['team_favorite'] != team_name else (row['point_differential'] - float(row['spread_line'])), axis=1)
    games_df['spread_covered'] = games_df['ats_differential'] > 0
    
    return games_df

# Function to summarize ATS record
def summarize_ats_record(team_name, seasons, db_path):
    # Get the games where the team was the underdog
    underdog_games_df = get_underdog_games(team_name, seasons, db_path)
    
    # Calculate ATS record
    ats_record = underdog_games_df['spread_covered'].sum()
    total_games = len(underdog_games_df)
    
    # Summarize the results
    summary = underdog_games_df[['season', 'week', 'away_team', 'home_team', 'spread_line', 'team_favorite', 'result', 'home_score', 'away_score', 'ats_differential', 'spread_covered']]
    
    # Return the results and ATS record
    return summary, ats_record, total_games

# Function to print the game summary with ATS result and adjusted point differential (relative to the spread)
def print_game_summary_with_ats_and_diff(team_name, seasons, db_path):
    summary_df, ats_record, total_games = summarize_ats_record(team_name, seasons, db_path)

    result_str = ""
    for season in seasons:
        season_games = summary_df[summary_df['season'] == season]
        if not season_games.empty:
            result_str += f"\n{season}:\n"
            for _, game in season_games.iterrows():
                # Determine the winner and loser
                result = "WIN" if game['ats_differential'] > 0 else "LOSE"
                
                # Determine ATS result
                ats_result = "WIN" if game['spread_covered'] else "LOSE"
                
                # Determine ATS differential with sign
                ats_diff = f"+{game['ats_differential']}" if game['ats_differential'] > 0 else f"{game['ats_differential']}"
                
                # Format home and away teams
                matchup = f"{game['away_team']} @ {game['home_team']}"
                if game['home_team'] == team_name:
                    matchup = f"{game['home_team']} vs {game['away_team']}"
                
                # Construct the result string
                # result_str += f"Week {game['week']}: {matchup}, spread: {game['spread_line']} ({game['team_favorite']} favored), result: {team_name} {result} {game['away_score']}-{game['home_score']} (ATS Diff: {ats_diff}). ATS: {ats_result}\n"
                result_str += f"Week {game['week']}: {matchup}, spread: {game['spread_line']} ({game['team_favorite']} favored), result: {team} {result} {game['away_score']}-{game['home_score']}, STRAIGHT-UP: {result}, ATS: {ats_result}, ATS Diff: {ats_diff})\n"

    # Print the game summary and ATS record
    result_str += f"\nATS Record: {int(ats_record)}-{total_games - int(ats_record)}\n"
    result_str += f"Straight-up Record: {int(summary_df['ats_differential'].gt(0).sum())}-{int(summary_df['ats_differential'].le(0).sum())}\n"
    
    return result_str
# Example usage
db_path = 'data/nfl.db'  # Update this with the correct path to your database file
team = 'DAL'  # Example: Dallas Cowboys
seasons = [2021, 2022, 2023]  # Specify the seasons you want to analyze

# Print the game summary for the team
print(print_game_summary_with_ats_and_diff(team, seasons, db_path))



2021:
Week 01: DAL @ TB, spread: 10.0 (TB favored), result: DAL WIN 29.0-31.0, STRAIGHT-UP: WIN, ATS: WIN, ATS Diff: +8.0)
Week 02: DAL @ LAC, spread: 3.0 (LAC favored), result: DAL WIN 20.0-17.0, STRAIGHT-UP: WIN, ATS: WIN, ATS Diff: +6.0)
Week 08: DAL @ MIN, spread: 4.5 (MIN favored), result: DAL WIN 20.0-16.0, STRAIGHT-UP: WIN, ATS: WIN, ATS Diff: +8.5)
Week 11: DAL @ KC, spread: 2.5 (KC favored), result: DAL LOSE 9.0-19.0, STRAIGHT-UP: LOSE, ATS: LOSE, ATS Diff: -7.5)

2022:
Week 01: DAL vs TB, spread: -2.5 (TB favored), result: DAL LOSE 19.0-3.0, STRAIGHT-UP: LOSE, ATS: LOSE, ATS Diff: -18.5)
Week 02: DAL vs CIN, spread: -7.5 (CIN favored), result: DAL LOSE 17.0-20.0, STRAIGHT-UP: LOSE, ATS: LOSE, ATS Diff: -4.5)
Week 03: DAL @ NYG, spread: 1.0 (NYG favored), result: DAL WIN 23.0-16.0, STRAIGHT-UP: WIN, ATS: WIN, ATS Diff: +8.0)
Week 05: DAL @ LAR, spread: 5.5 (LAR favored), result: DAL WIN 22.0-10.0, STRAIGHT-UP: WIN, ATS: WIN, ATS Diff: +17.5)
Week 06: DAL @ PHI, spread: 6.5 (P

In [28]:
# Same with better printouts ^

# Function to get underdog games
def get_underdog_games(team_name, seasons, db_path):
    # Connect to the database
    conn = sqlite3.connect(db_path)
    
    # Prepare a query to get all games for the given team where they were the underdog
    query = f"""
    SELECT season, week, away_team, home_team, spread_line, team_favorite, result, home_score, away_score 
    FROM Games
    WHERE season IN ({','.join(map(str, seasons))})
    AND (home_team = '{team_name}' OR away_team = '{team_name}')
    AND team_favorite != '{team_name}';
    """
    
    # Execute query
    games = conn.execute(query).fetchall()
    conn.close()
    
    # Create a DataFrame
    columns = ['season', 'week', 'away_team', 'home_team', 'spread_line', 'team_favorite', 'result', 'home_score', 'away_score']
    games_df = pd.DataFrame(games, columns=columns)
    
    # Calculate the point differential and whether the team covered the spread
    games_df['point_differential'] = games_df.apply(lambda row: row['away_score'] - row['home_score'] if row['away_team'] == team_name else row['home_score'] - row['away_score'], axis=1)
    games_df['ats_differential'] = games_df.apply(lambda row: (row['point_differential'] + float(row['spread_line'])) if row['team_favorite'] != team_name else (row['point_differential'] - float(row['spread_line'])), axis=1)
    games_df['spread_covered'] = games_df['ats_differential'] > 0
    
    return games_df

# Function to summarize ATS record
def summarize_ats_record(team_name, seasons, db_path):
    # Get the games where the team was the underdog
    underdog_games_df = get_underdog_games(team_name, seasons, db_path)
    
    # Calculate ATS record
    ats_record = underdog_games_df['spread_covered'].sum()
    total_games = len(underdog_games_df)
    
    # Summarize the results
    summary = underdog_games_df[['season', 'week', 'away_team', 'home_team', 'spread_line', 'team_favorite', 'result', 'home_score', 'away_score', 'ats_differential', 'spread_covered']]
    
    # Return the results and ATS record
    return summary, ats_record, total_games

# Function to print the game summary with ATS result and adjusted point differential (relative to the spread)
def print_game_summary_with_ats_and_diff(team_name, seasons, db_path):
    summary_df, ats_record, total_games = summarize_ats_record(team_name, seasons, db_path)

    result_str = ""
    for season in seasons:
        season_games = summary_df[summary_df['season'] == season]
        if not season_games.empty:
            result_str += f"\n          {season}:\n"
            result_str += '*' * 25 + '\n'
            for _, game in season_games.iterrows():
                # Determine the winner and loser
                result = "WIN" if game['ats_differential'] > 0 else "LOSE"
                
                # Determine ATS result
                ats_result = "WIN" if game['spread_covered'] else "LOSE"
                
                # Determine ATS differential with sign
                ats_diff = f"+{game['ats_differential']}" if game['ats_differential'] > 0 else f"{game['ats_differential']}"
                
                # Format home and away teams
                matchup = f"{game['away_team']} @ {game['home_team']}"
                if game['home_team'] == team_name:
                    matchup = f"{game['home_team']} vs {game['away_team']}"
                
                # Construct the result string
                # result_str += f"Week {game['week']}: {matchup}, spread: {game['spread_line']} ({game['team_favorite']} favored), result: {team_name} {result} {game['away_score']}-{game['home_score']} (ATS Diff: {ats_diff}). ATS: {ats_result}\n"
                # result_str += f"Week {game['week']}: {matchup}, spread: {game['spread_line']} ({game['team_favorite']} favored), result: {team} {result} {game['away_score']}-{game['home_score']}, STRAIGHT-UP: {result}, ATS: {ats_result}, ATS Diff: {ats_diff})\n"

              # Construct the result string across multiple lines
                result_str += f"{game['away_team']} {game['away_score']} @ {game['home_team']} {game['home_score']}\n"
                result_str += f"{game['away_team']} Spread: {'+' + str(game['spread_line']) if game['team_favorite'] == game['home_team'] else '-' + str(game['spread_line'])}\n"
                result_str += f"{game['home_team']} Spread: {'+' + str(game['spread_line']) if game['team_favorite'] == game['away_team'] else '-' + str(game['spread_line'])}\n"
                result_str += f"{game['away_team']} Score: {game['away_score']}\n"
                result_str += f"{game['home_team']} Score: {game['home_score']}\n"
                # result_str += f"STRAIGHT-UP: {'WIN' if game['point_differential'] > 0 else 'LOSE'}, "
                result_str += f"ATS: {'WIN' if game['spread_covered'] else 'LOSE'}, "
                result_str += f"ATS Diff: {f'+{game['ats_differential']}' if game['ats_differential'] > 0 else game['ats_differential']}\n"
                result_str += "-"*30 + "\n"  # Separator for readability

            result_str += '\n'
            # result_str += '*' * 50 + '\n\n\n'

    # Print the game summary and ATS record
    result_str += f"\nATS Record: {int(ats_record)}-{total_games - int(ats_record)}\n"
    result_str += f"Straight-up Record: {int(summary_df['ats_differential'].gt(0).sum())}-{int(summary_df['ats_differential'].le(0).sum())}\n"
    return result_str


# Example usage
db_path = 'data/nfl.db'  # Update this with the correct path to your database file
team = 'DAL'  # Example: Dallas Cowboys
seasons = [2021, 2022, 2023]  # Specify the seasons you want to analyze

# Print the game summary for the team
print(print_game_summary_with_ats_and_diff(team, seasons, db_path))



          2021:
*************************
DAL 29.0 @ TB 31.0
DAL Spread: +10.0
TB Spread: -10.0
DAL Score: 29.0
TB Score: 31.0
ATS: WIN, ATS Diff: +8.0
------------------------------
DAL 20.0 @ LAC 17.0
DAL Spread: +3.0
LAC Spread: -3.0
DAL Score: 20.0
LAC Score: 17.0
ATS: WIN, ATS Diff: +6.0
------------------------------
DAL 20.0 @ MIN 16.0
DAL Spread: +4.5
MIN Spread: -4.5
DAL Score: 20.0
MIN Score: 16.0
ATS: WIN, ATS Diff: +8.5
------------------------------
DAL 9.0 @ KC 19.0
DAL Spread: +2.5
KC Spread: -2.5
DAL Score: 9.0
KC Score: 19.0
ATS: LOSE, ATS Diff: -7.5
------------------------------


          2022:
*************************
TB 19.0 @ DAL 3.0
TB Spread: --2.5
DAL Spread: +-2.5
TB Score: 19.0
DAL Score: 3.0
ATS: LOSE, ATS Diff: -18.5
------------------------------
CIN 17.0 @ DAL 20.0
CIN Spread: --7.5
DAL Spread: +-7.5
CIN Score: 17.0
DAL Score: 20.0
ATS: LOSE, ATS Diff: -4.5
------------------------------
DAL 23.0 @ NYG 16.0
DAL Spread: +1.0
NYG Spread: -1.0
DAL Score:

In [29]:
# Same but all 32 teams and saving to files ^

# Function to get underdog games
def get_underdog_games(team_name, seasons, db_path):
    # Connect to the database
    conn = sqlite3.connect(db_path)
    
    # Prepare a query to get all games for the given team where they were the underdog
    query = f"""
    SELECT season, week, away_team, home_team, spread_line, team_favorite, result, home_score, away_score 
    FROM Games
    WHERE season IN ({','.join(map(str, seasons))})
    AND (home_team = '{team_name}' OR away_team = '{team_name}')
    AND team_favorite != '{team_name}';
    """
    
    # Execute query
    games = conn.execute(query).fetchall()
    conn.close()
    
    # Create a DataFrame
    columns = ['season', 'week', 'away_team', 'home_team', 'spread_line', 'team_favorite', 'result', 'home_score', 'away_score']
    games_df = pd.DataFrame(games, columns=columns)
    
    # Calculate the point differential and whether the team covered the spread
    games_df['point_differential'] = games_df.apply(lambda row: row['away_score'] - row['home_score'] if row['away_team'] == team_name else row['home_score'] - row['away_score'], axis=1)
    games_df['ats_differential'] = games_df.apply(lambda row: (row['point_differential'] + float(row['spread_line'])) if row['team_favorite'] != team_name else (row['point_differential'] - float(row['spread_line'])), axis=1)
    games_df['spread_covered'] = games_df['ats_differential'] > 0
    
    return games_df

# Function to summarize ATS record and return data for CSV
def summarize_ats_record_for_csv(team_name, seasons, db_path):
    underdog_games_df = get_underdog_games(team_name, seasons, db_path)
    
    results = []
    
    for _, game in underdog_games_df.iterrows():
        # Construct the result as a dictionary for later CSV storage
        result = {
            'season': game['season'],
            'week': game['week'],
            'team': team_name,
            'opponent': game['home_team'] if game['away_team'] == team_name else game['away_team'],
            'team_score': game['away_score'] if game['away_team'] == team_name else game['home_score'],
            'opponent_score': game['home_score'] if game['away_team'] == team_name else game['away_score'],
            'spread': game['spread_line'],
            'team_favorite': game['team_favorite'],
            'ats_result': 'WIN' if game['spread_covered'] else 'LOSE',
            'ats_differential': game['ats_differential'],
            'straight_up_result': 'WIN' if game['point_differential'] > 0 else 'LOSE'
        }
        results.append(result)
    
    return results

# Function to save the results for all teams into a CSV
def save_all_teams_to_csv(teams, seasons, db_path, csv_file_path):
    all_results = []
    for team in teams:
        team_results = summarize_ats_record_for_csv(team, seasons, db_path)
        all_results.extend(team_results)  # Accumulate the results for all teams
    
    # Convert the list of results to a DataFrame
    df = pd.DataFrame(all_results)
    
    # Save DataFrame to CSV
    df.to_csv(csv_file_path, index=False)
    print(f"Data saved to {csv_file_path}")

# Example usage
db_path = 'data/nfl.db'  # Update this with the correct path to your database file
teams = [
    'ARI', 'ATL', 'BAL', 'BUF', 'CAR', 'CHI', 'CIN', 'CLE',
    'DAL', 'DEN', 'DET', 'GB', 'HOU', 'IND', 'JAX', 'KC',
    'LV', 'LAC', 'LAR', 'MIA', 'MIN', 'NE', 'NO', 'NYG',
    'NYJ', 'PHI', 'PIT', 'SF', 'SEA', 'TB', 'TEN', 'WAS'
]  # List of all NFL teams
seasons = [2021, 2022, 2023]  # Specify the seasons you want to analyze
csv_file_path = 'nfl_game_summaries.csv'  # Path to save the CSV file

# Save the results to a CSV
save_all_teams_to_csv(teams, seasons, db_path, csv_file_path)

Data saved to nfl_game_summaries.csv


In [30]:
!open nfl_game_summaries.csv

In [ ]:
# Games as favorites 
# Still testing

# Function to determine the ATS result for a game using actual scores and spread
def ats_result(row, team):
    try:
        spread = float(row['spread_line'])
        score_diff = row['home_score'] - row['away_score'] if row['home_team'] == team else row['away_score'] - row['home_score']
        if score_diff > spread:
            return 'Win'
        elif score_diff < spread:
            return 'Loss'
        else:
            return 'Push'
    except ValueError:
        return 'Invalid'

# Function to infer the favorite team based on the spread_line
def infer_favorite(row):
    return row['away_team'] if row['spread_line'] > 0 else row['home_team']

def get_underdog_games(team_name, seasons):
    # Connect to the database
    conn = sqlite3.connect('data/nfl.db')
    
    # Prepare a query to get all games for the given team
    query = f"""
    SELECT season, week, away_team, home_team, home_score, away_score, spread_line, result 
    FROM Games
    WHERE season IN ({','.join(map(str, seasons))})
    AND (home_team = '{team_name}' OR away_team = '{team_name}');
    """
    
    # Execute query
    games = conn.execute(query).fetchall()
    conn.close()
    
    # Create a DataFrame
    columns = ['season', 'week', 'away_team', 'home_team', 'home_score', 'away_score', 'spread_line', 'result']
    games_df = pd.DataFrame(games, columns=columns)
    
    # Infer the favorite team based on the spread_line
    games_df['team_favorite'] = games_df.apply(infer_favorite, axis=1)
    
    # Filter for games where the team was the underdog
    games_df = games_df[games_df['team_favorite'] != team_name]
    
    # Calculate point differential
    games_df['point_differential'] = games_df.apply(lambda row: row['away_score'] - row['home_score'] if row['away_team'] == team_name else row['home_score'] - row['away_score'], axis=1)
    
    # Calculate ATS: Whether the team covered the spread
    games_df['spread_covered'] = games_df.apply(lambda row: (row['point_differential'] + float(row['spread_line'])) > 0, axis=1)
    
    # Calculate Straight-up: Whether the team won the game outright
    games_df['won_straight_up'] = games_df['point_differential'] > 0
    
    return games_df

def summarize_ats_record(team_name, seasons):
    # Get the games where the team was the underdog
    underdog_games_df = get_underdog_games(team_name, seasons)
    
    # Calculate ATS record
    ats_record = underdog_games_df['spread_covered'].sum()
    total_games = len(underdog_games_df)
    
    # Summarize the results
    summary = underdog_games_df[['season', 'week', 'away_team', 'home_team', 'spread_line', 'team_favorite', 'result', 'home_score', 'away_score', 'point_differential', 'spread_covered', 'won_straight_up']]
    
    # Return the results, ATS record, and straight-up record
    return summary, ats_record, total_games

def print_game_summary(team_name, seasons):
    summary_df, ats_record, total_games = summarize_ats_record(team_name, seasons)
    
    straight_up_wins = summary_df['won_straight_up'].sum()
    straight_up_losses = total_games - straight_up_wins
    
    result_str = ""
    for season in seasons:
        season_games = summary_df[summary_df['season'] == season]
        if not season_games.empty:
            result_str += f"\n{season}:\n"
            for _, game in season_games.iterrows():
                # Determine the winner and loser for Straight-up
                result = "won" if game['won_straight_up'] else "lost"
                
                # Format home and away teams
                matchup = f"{game['away_team']} @ {game['home_team']}"
                if game['home_team'] == team_name:
                    matchup = f"{game['home_team']} vs {game['away_team']}"
                
                # Construct the result string
                result_str += f"Week {game['week']}: {matchup}, spread: {game['spread_line']} ({game['team_favorite']} favored), result: {team_name} {result} {game['away_score']}-{game['home_score']}.\n"
    
    # Print the game summary and ATS record
    print(result_str)
    print(f"ATS Record: {int(ats_record)}-{total_games - int(ats_record)}")
    print(f"Straight-up Record: {int(straight_up_wins)}-{int(straight_up_losses)}")

# Usage
team = 'DAL'  # You can change this to any team, e.g., 'SF' for San Francisco
seasons = [2021, 2022, 2023]  # Specify the seasons you want to query

# Print the game summary
print_game_summary(team, seasons)

In [ ]:
import sqlite3
import pandas as pd

def get_underdog_games(team_name, seasons):
    # Connect to the database
    conn = sqlite3.connect('data/nfl.db')
    
    # Prepare a query to get all games for the given team where they were the underdog
    query = f"""
    SELECT season, week, away_team, home_team, spread_line, team_favorite, result, home_score, away_score 
    FROM Games
    WHERE season IN ({','.join(map(str, seasons))})
    AND (home_team = '{team_name}' OR away_team = '{team_name}')
    AND team_favorite != '{team_name}';
    """
    
    # Execute query
    games = conn.execute(query).fetchall()
    conn.close()
    
    # Create a DataFrame
    columns = ['season', 'week', 'away_team', 'home_team', 'spread_line', 'team_favorite', 'result', 'home_score', 'away_score']
    games_df = pd.DataFrame(games, columns=columns)
    
    # Calculate point differential
    games_df['point_differential'] = games_df.apply(lambda row: row['away_score'] - row['home_score'] if row['away_team'] == team_name else row['home_score'] - row['away_score'], axis=1)
    
    # Calculate ATS: Whether the team covered the spread
    games_df['spread_covered'] = games_df.apply(lambda row: (row['point_differential'] + float(row['spread_line'])) > 0, axis=1)
    
    # Calculate Straight-up: Whether the team won the game outright
    games_df['won_straight_up'] = games_df['point_differential'] > 0
    
    return games_df

def summarize_ats_record(team_name, seasons):
    # Get the games where the team was the underdog
    underdog_games_df = get_underdog_games(team_name, seasons)
    
    # Calculate ATS record
    ats_record = underdog_games_df['spread_covered'].sum()
    total_games = len(underdog_games_df)
    
    # Summarize the results
    summary = underdog_games_df[['season', 'week', 'away_team', 'home_team', 'spread_line', 'team_favorite', 'result', 'home_score', 'away_score', 'point_differential', 'spread_covered', 'won_straight_up']]
    
    # Return the results, ATS record, and straight-up record
    return summary, ats_record, total_games

def print_game_summary(team_name, seasons):
    summary_df, ats_record, total_games = summarize_ats_record(team_name, seasons)
    
    straight_up_wins = summary_df['won_straight_up'].sum()
    straight_up_losses = total_games - straight_up_wins
    
    result_str = ""
    for season in seasons:
        season_games = summary_df[summary_df['season'] == season]
        if not season_games.empty:
            result_str += f"\n{season}:\n"
            for _, game in season_games.iterrows():
                # Determine the winner and loser for Straight-up
                result = "won" if game['won_straight_up'] else "lost"
                
                # Format home and away teams
                matchup = f"{game['away_team']} @ {game['home_team']}"
                if game['home_team'] == team_name:
                    matchup = f"{game['home_team']} vs {game['away_team']}"
                
                # Construct the result string
                result_str += f"Week {game['week']}: {matchup}, spread: {game['spread_line']} ({game['team_favorite']} favored), result: {team_name} {result} {game['away_score']}-{game['home_score']}.\n"
    
    # Print the game summary and ATS record
    print(result_str)
    print(f"ATS Record: {int(ats_record)}-{total_games - int(ats_record)}")
    print(f"Straight-up Record: {int(straight_up_wins)}-{int(straight_up_losses)}")

# Usage
team = 'DAL'  # You can change this to any team, e.g., 'SF' for San Francisco
seasons = [2021, 2022, 2023]  # Specify the seasons you want to query

# Print the game summary
print_game_summary(team, seasons)


In [ ]:
# --- Creating home_spread and away_spread columns in nfl.db --- #

# Step 1: Connect to the SQLite database
conn = sqlite3.connect('data/nfl.db')
cursor = conn.cursor()

# Step 2: Read the data from the Games table into a Pandas DataFrame
games_df = pd.read_sql_query("SELECT * FROM Games WHERE season != 2024", conn)

# Step 3: Calculate home_spread and away_spread based on the spread_line
games_df['home_spread'] = games_df['spread_line'].apply(lambda x: f"+{x}" if x > 0 else f"{x}")
games_df['away_spread'] = games_df['spread_line'].apply(lambda x: f"-{x}" if x > 0 else f"+{-x}")

# Step 4: Add the new columns to the database
cursor.execute("ALTER TABLE Games ADD COLUMN home_spread TEXT;")
cursor.execute("ALTER TABLE Games ADD COLUMN away_spread TEXT;")

# Step 5: Update the database with the calculated values
for index, row in games_df.iterrows():
    cursor.execute("""
        UPDATE Games
        SET home_spread = ?, away_spread = ?
        WHERE game_id = ?
    """, (row['home_spread'], row['away_spread'], row['game_id']))

# Step 6: Commit the changes and close the connection
conn.commit()
conn.close()

print("Columns 'home_spread' and 'away_spread' have been added and updated in the 'Games' table of 'nfl.db'.")


In [ ]:
import sqlite3
import pandas as pd

# Function to determine the ATS result for a game using actual scores and spread
def ats_result(row, team):
    try:
        spread = float(row['home_spread']) if row['home_team'] == team else float(row['away_spread'])
        score_diff = row['home_score'] - row['away_score'] if row['home_team'] == team else row['away_score'] - row['home_score']
        if score_diff > spread:
            return 'Win'
        elif score_diff < spread:
            return 'Loss'
        else:
            return 'Push'
    except ValueError:
        return 'Invalid'

def get_underdog_games(team_name, seasons):
    # Connect to the database
    conn = sqlite3.connect('data/nfl.db')
    
    # Prepare a query to get all games for the given team where they were the underdog
    query = f"""
    SELECT season, week, away_team, home_team, home_score, away_score, home_spread, away_spread, team_favorite, result 
    FROM Games
    WHERE season IN ({','.join(map(str, seasons))})
    AND (home_team = '{team_name}' OR away_team = '{team_name}')
    AND team_favorite != '{team_name}';
    """
    
    # Execute query
    games = conn.execute(query).fetchall()
    conn.close()
    
    # Create a DataFrame
    columns = ['season', 'week', 'away_team', 'home_team', 'home_score', 'away_score', 'home_spread', 'away_spread', 'team_favorite', 'result']
    games_df = pd.DataFrame(games, columns=columns)
    
    # Convert relevant columns to numeric, handling errors
    games_df['home_spread'] = pd.to_numeric(games_df['home_spread'], errors='coerce')
    games_df['away_spread'] = pd.to_numeric(games_df['away_spread'], errors='coerce')
    
    # Calculate ATS result
    games_df['ATS_Result'] = games_df.apply(lambda row: ats_result(row, team_name), axis=1)
    
    return games_df

def summarize_ats_record(team_name, seasons):
    # Get the games where the team was the underdog
    underdog_games_df = get_underdog_games(team_name, seasons)
    
    # Calculate ATS record
    overall_ats_record = underdog_games_df['ATS_Result'].value_counts()
    ats_record_win = overall_ats_record.get('Win', 0)
    ats_record_loss = overall_ats_record.get('Loss', 0)
    ats_record_push = overall_ats_record.get('Push', 0)
    
    # Summarize the results
    summary = underdog_games_df[['season', 'week', 'away_team', 'home_team', 'home_score', 'away_score', 'home_spread', 'away_spread', 'ATS_Result']]
    
    # Return the results and ATS record
    return summary, ats_record_win, ats_record_loss, ats_record_push

def print_game_summary(team_name, seasons):
    summary_df, ats_wins, ats_losses, ats_pushes = summarize_ats_record(team_name, seasons)
    
    result_str = ""
    for season in seasons:
        season_games = summary_df[summary_df['season'] == season]
        if not season_games.empty:
            result_str += f"\n{season}:\n"
            for _, game in season_games.iterrows():
                # Determine the ATS result
                result = game['ATS_Result']
                
                # Format home and away teams
                matchup = f"{game['away_team']} @ {game['home_team']}"
                if game['home_team'] == team_name:
                    matchup = f"{game['home_team']} vs {game['away_team']}"
                
                # Construct the result string
                result_str += f"Week {game['week']}: {matchup}, spread: {game['home_spread'] if game['home_team'] == team_name else game['away_spread']} ({game['team_favorite']} favored), result: {result}, score: {game['away_score']}-{game['home_score']}.\n"
    
    # Print the game summary and ATS record
    print(result_str)
    print(f"ATS Record: {ats_wins}-{ats_losses}-{ats_pushes}")

# Usage
team = 'DAL'  # You can change this to any team, e.g., 'SF' for San Francisco
seasons = [2021, 2022, 2023]  # Specify the seasons you want to query

# Print the game summary
print_game_summary(team, seasons)


In [ ]:
import sqlite3
import pandas as pd

# Function to determine the ATS result for a game using actual scores and spread
def ats_result(row, team):
    try:
        spread = float(row['home_spread']) if row['home_team'] == team else float(row['away_spread'])
        score_diff = row['home_score'] - row['away_score'] if row['home_team'] == team else row['away_score'] - row['home_score']
        if score_diff > spread:
            return 'Win'
        elif score_diff < spread:
            return 'Loss'
        else:
            return 'Push'
    except ValueError:
        return 'Invalid'

# Function to infer the favorite team based on the spread line
def infer_favorite(row):
    if float(row['spread_line']) < 0:
        return row['home_team']
    else:
        return row['away_team']

def get_underdog_games(team_name, seasons):
    # Connect to the database
    conn = sqlite3.connect('data/nfl.db')
    
    # Prepare a query to get all games for the given team
    query = f"""
    SELECT season, week, away_team, home_team, home_score, away_score, spread_line, home_spread, away_spread
    FROM Games
    WHERE season IN ({','.join(map(str, seasons))})
    AND (home_team = '{team_name}' OR away_team = '{team_name}');
    """
    
    # Execute query
    games = conn.execute(query).fetchall()
    conn.close()
    
    # Create a DataFrame
    columns = ['season', 'week', 'away_team', 'home_team', 'home_score', 'away_score', 'spread_line', 'home_spread', 'away_spread']
    games_df = pd.DataFrame(games, columns=columns)
    
    # Convert relevant columns to numeric, handling errors
    games_df['home_spread'] = pd.to_numeric(games_df['home_spread'], errors='coerce')
    games_df['away_spread'] = pd.to_numeric(games_df['away_spread'], errors='coerce')
    
    # Infer the favorite team based on the spread line
    games_df['team_favorite'] = games_df.apply(infer_favorite, axis=1)
    
    # Filter for games where the team was the underdog
    games_df = games_df[games_df['team_favorite'] != team_name]
    
    # Calculate ATS result
    games_df['ATS_Result'] = games_df.apply(lambda row: ats_result(row, team_name), axis=1)
    
    return games_df

def summarize_ats_record(team_name, seasons):
    # Get the games where the team was the underdog
    underdog_games_df = get_underdog_games(team_name, seasons)
    
    # Calculate ATS record
    overall_ats_record = underdog_games_df['ATS_Result'].value_counts()
    ats_record_win = overall_ats_record.get('Win', 0)
    ats_record_loss = overall_ats_record.get('Loss', 0)
    ats_record_push = overall_ats_record.get('Push', 0)
    
    # Summarize the results
    summary = underdog_games_df[['season', 'week', 'away_team', 'home_team', 'home_score', 'away_score', 'home_spread', 'away_spread', 'ATS_Result']]
    
    # Return the results and ATS record
    return summary, ats_record_win, ats_record_loss, ats_record_push

def print_game_summary(team_name, seasons):
    summary_df, ats_wins, ats_losses, ats_pushes = summarize_ats_record(team_name, seasons)
    
    result_str = ""
    for season in seasons:
        season_games = summary_df[summary_df['season'] == season]
        if not season_games.empty:
            result_str += f"\n{season}:\n"
            for _, game in season_games.iterrows():
                # Determine the ATS result
                result = game['ATS_Result']
                
                # Format home and away teams
                matchup = f"{game['away_team']} @ {game['home_team']}"
                if game['home_team'] == team_name:
                    matchup = f"{game['home_team']} vs {game['away_team']}"
                
                # Construct the result string
                result_str += f"Week {game['week']}: {matchup}, spread: {game['home_spread'] if game['home_team'] == team_name else game['away_spread']}, result: {result}, score: {game['away_score']}-{game['home_score']}.\n"
    
    # Print the game summary and ATS record
    print(result_str)
    print(f"ATS Record: {ats_wins}-{ats_losses}-{ats_pushes}")

# Usage
team = 'DAL'  # You can change this to any team, e.g., 'SF' for San Francisco
seasons = [2021, 2022, 2023]  # Specify the seasons you want to query

# Print the game summary
print_game_summary(team, seasons)


In [16]:
# Remove unplayed games from all_team_game_logs.csv

# Load the 2024 season data
df_2024 = pd.read_csv('data/all_team_game_logs.csv')

# Filter for unplayed games where the game_id contains "2024" and both home and away points are either NaN or 0
unplayed_games = df_2024[
    df_2024['game_id'].str.contains('2024') &  # Check if 'game_id' contains "2024"
    ((df_2024['home_pts_off'].isnull() | (df_2024['home_pts_off'] == 0)) &
     (df_2024['away_pts_off'].isnull() | (df_2024['away_pts_off'] == 0)))
]

# Create a list of unplayed game_ids
unplayed_game_ids = unplayed_games['game_id'].tolist()

# Output the list of unplayed game_ids
print("List of unplayed 2024 game_ids:")
print(unplayed_game_ids)

# Filter out the unplayed games from the original DataFrame
df_filtered = df_2024[~df_2024['game_id'].isin(unplayed_game_ids)]

# Save the filtered DataFrame back to the same CSV (or a new one if desired)
# df_filtered.to_csv('data/all_team_game_logs.csv', index=False)

# print("Unplayed games removed and updated CSV saved.")

List of unplayed 2024 game_ids:
['2024_03_BAL_DAL', '2024_03_CAR_LVR', '2024_03_CHI_IND', '2024_03_DEN_TB', '2024_03_DET_ARI', '2024_03_GB_TEN', '2024_03_HOU_MIN', '2024_03_JAX_BUF', '2024_03_KC_ATL', '2024_03_LAC_PIT', '2024_03_MIA_SEA', '2024_03_NE_NYJ', '2024_03_NYG_CLE', '2024_03_PHI_NO', '2024_03_SF_LAR', '2024_03_WAS_CIN', '2024_04_BUF_BAL', '2024_04_CIN_CAR', '2024_04_CLE_LVR', '2024_04_DAL_NYG', '2024_04_DEN_NYJ', '2024_04_JAX_HOU', '2024_04_KC_LAC', '2024_04_LAR_CHI', '2024_04_MIN_GB', '2024_04_NE_SF', '2024_04_NO_ATL', '2024_04_PHI_TB', '2024_04_PIT_IND', '2024_04_SEA_DET', '2024_04_TEN_MIA', '2024_04_WAS_ARI', '2024_05_ARI_SF', '2024_05_BAL_CIN', '2024_05_BUF_HOU', '2024_05_CAR_CHI', '2024_05_CLE_WAS', '2024_05_DAL_PIT', '2024_05_GB_LAR', '2024_05_IND_JAX', '2024_05_LVR_DEN', '2024_05_MIA_NE', '2024_05_NO_KC', '2024_05_NYG_SEA', '2024_05_NYJ_MIN', '2024_05_TB_ATL', '2024_06_ARI_GB', '2024_06_ATL_CAR', '2024_06_BUF_NYJ', '2024_06_CIN_NYG', '2024_06_CLE_PHI', '2024_06_DET_DAL'